### LLM Setup

In [2]:
#from langchain_community.llms import Ollama
from langchain_community.embeddings import GPT4AllEmbeddings

gpt4all_embd = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf")
#llm = Ollama(model="llama2", temperature=0.5)

from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", nvidia_api_key="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")

llm = ChatNVIDIA(model="meta/llama3-70b-instruct", nvidia_api_key="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", temperature=0.45)

d:\Installs\Miniconda\envs\LLM\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:556: UserWarning: Found nvidia/embed-qa-4 in available_models, but type is unknown and inference may fail.
  warnings.warn(


### Utility functions

In [3]:
async def astream(invocable, inputs, *args, **kwargs):
    async for chunk in invocable.astream(inputs, *args, **kwargs):
        print(chunk, end="")

def parse_bullet_points(text):
    bullet_char = '•'
    lines = text.split('\n')

    lines = list(filter(lambda line: line.startswith(bullet_char), lines))
    return text
    return "\n".join(lines)

#### Prompt Chains

In [18]:
from langchain_core.prompts import ChatPromptTemplate

code_interpretation_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are an AI agent tasked with understanding chunks from a codebase that is written in C++ and CUDA. Keep the following points in mind while writing interpretation:
• Note down and briefly describe any headers, macros, or #defines present and their purposes.
• Identify and list the class names, along with all member functions and variables in the chunk of code.
• For each function, note its purpose and role within the code.
• For each member variable of a class, note where it is referred to and its purpose.
• All responses MUST be in bullet points - must start with the character '•'
• Do NOT use any other sentence formatting.
• Do NOT include any introductory or concluding statements."""),
    ("user", "The code chunk you will need to process is:\n{code_chunk}")
])

code_interpretation_chain = code_interpretation_prompt_template | llm

In [19]:
from langchain_core.prompts import ChatPromptTemplate

combined_interpretation_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are an AI agent tasked with combining the essence of the below two code inferences:
• Combine the essence of two code inferences into a single, standalone passage.
• Treat function or class interpretations that spill over as a single logical unit.
• Include all unique information from both summaries.
• Use concise technical language.
• Ensure consistency in terminology and formatting.
• All responses MUST be in bullet points - must start with the character '•'
• Do NOT use any other sentence formatting.
• Do NOT include any introductory or concluding statements.
"""),
    ("user", "The two code inferences you will need to cobine are:\n text 1:{summary_1} \n text 2:{summary_2}")
])

combined_interpretation_chain = combined_interpretation_prompt_template | llm

#### Vectorstore database generation

In [23]:
import os, os.path as osp
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language
from langchain_community.document_loaders import TextLoader
from langchain.docstore.document import Document
from tqdm import tqdm
import glob

class KnowledgeBase:
    def __init__(self, repo_path):
        self.db = Chroma(embedding_function=gpt4all_embd, persist_directory=".\\knowledge_db\\vector_store")
        self.language_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.CPP, chunk_size=1000, chunk_overlap=100)
        self.repo_path = repo_path
        self.in_memory_db = []
        self.per_file_interpretations = []

    def add_knowledge_per_file(self, file_path):
        print(f"Processing {file_path}...")
        interpretations_per_chunk = []
        loader = TextLoader(file_path)
        file_data = loader.load()
        code_chunks = self.language_splitter.split_documents(file_data)

        for chunk in tqdm(code_chunks):
            interpretation = code_interpretation_chain.invoke(chunk.page_content)
            interpretation_pointwise = parse_bullet_points(interpretation.content)
            print("****************************")
            print(interpretation_pointwise)
            print("****************************")
            source_file_name = chunk.metadata["source"]
            interpretations_per_chunk.append(Document(page_content=source_file_name+"\n"+interpretation_pointwise, metadata=chunk.metadata))
        
        read_buffer = interpretations_per_chunk.copy()
        merge_buffer = []

        while len(read_buffer) > 1:
            for i in range(0, len(read_buffer), 2):
                if i == len(read_buffer)-1:
                    merge_buffer.append(Document(page_content=read_buffer[i].page_content, metadata=read_buffer[i].metadata))
                else:
                    interpretation = combined_interpretation_chain.invoke({"summary_1":read_buffer[i].page_content, "summary_2":read_buffer[i+1].page_content})
                    interpretation_pointwise = parse_bullet_points(interpretation.content)
                    source_file = read_buffer[i].metadata["source"]
                    print("****************************")
                    print(interpretation_pointwise)
                    print("****************************")
                    merge_buffer.append(Document(page_content=source_file+"\n"+interpretation_pointwise, metadata=read_buffer[i].metadata))

            read_buffer.clear()
            read_buffer.extend(merge_buffer)
            self.in_memory_db.extend(merge_buffer)
            merge_buffer.clear()

        self.per_file_interpretations.append(read_buffer[0])

    def generate_knowledge_base(self):
        
        source_files = glob.glob(self.repo_path + "\\*.*")

        for source_file in source_files:
            self.add_knowledge_per_file(source_file)
        
        read_buffer = self.per_file_interpretations.copy()
        merge_buffer = []

        while len(read_buffer) > 1:
            for i in range(0, len(read_buffer), 2):
                if i == len(read_buffer)-1:
                    merge_buffer.append(Document(page_content=read_buffer[i].page_content, metadata=read_buffer[i].metadata))
                else:
                    interpretation = combined_interpretation_chain.invoke({"summary_1":read_buffer[i].page_content, "summary_2":read_buffer[i+1].page_content})
                    interpretation_pointwise = parse_bullet_points(interpretation.content)
                    source_list_set = set(read_buffer[i].metadata["source"].split("|")).union(read_buffer[i+1].metadata["source"].split("|"))
                    source_list_str = "|".join(source_list_set)
                    combined_meta = {"source": source_list_str}

                    merge_buffer.append(Document(page_content=source_list_str+"\n"+interpretation_pointwise, metadata=combined_meta))

            read_buffer.clear()
            read_buffer.extend(merge_buffer)
            self.in_memory_db.extend(merge_buffer)
            merge_buffer.clear()

        import pickle

        with open(".\\temp\\in_memory_db_new.pkl", "wb") as pkl_file:
            pickle.dump(self.in_memory_db, pkl_file)
        self.db.add_documents(self.in_memory_db)

In [ ]:
repo_path = ".\\PathTracerAP"

knowledge_base = KnowledgeBase(repo_path)
knowledge_base.generate_knowledge_base()

In [34]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

qa_chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant that has received a query from a user related to a code base that you have understood thoroughly.You are also well-versed in c++ and CUDA. The query might involve details such as what a specific function does, the purpose of a line of code, how a component fits into the architecture, or any other technical aspect. You will be given a supporting context to answer the query."),
    ("human", "Answer the query below using the provided context:\nQuery:{query}\nContext:{context}")
])

threshold_score = 0.06
info_retriever = store_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': 10, 'score_threshold': threshold_score}
)
qa_chain = ({"query": RunnablePassthrough(), "context": info_retriever } | qa_chat_prompt_template | llm)
qa_chain.invoke("Enlist all the classes in the project").content

d:\Installs\Miniconda\envs\LLM\Lib\site-packages\langchain_core\vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='.\\PathTracerAP\\GPUKernels.cuh\n• `#pragma once`: This directive is used to indicate that the header file should only be included once in a single translation unit, avoiding multiple inclusions and potential compilation errors.\n\n• `#define GLM_FORCE_CUDA`: This macro is used to enable CUDA support in the GLM (Graphics Library Manager) library.\n\n• Headers included:\n  • <cuda.h>\n  • <cuda_runtime_api.h>\n  • "device_launch_parameters.h"\n  • <glm/glm.hpp>\n  • <glm/gtc/matrix_transform.hpp>\n  • <thrust/execution_policy.h>\n  • <thrust/random.h>\n  • <thrust/remove.h>\n  • <thrust/device_vector.h>\n  • <thrust/partition.h>\n  • <iostream>\n  • <chrono>\n  • "Renderer.h"\n  • "Primitive.h"\n  • "utility.h"\n\n• Macros/#defines present:\n  • FLOAT_MIN\n  • FLOAT_MAX\n  • GLM_FORCE_CUDA\n\n• Function:\n  • computeRayB

'I apologize, but the provided context is empty. To enlist all the classes in the project, I would need more information about the project, such as the code files or directories. \n\nIf you could provide me with the actual code or a description of the project, I would be happy to assist you in identifying the classes present in the project.'

### CodeDocBot - UI

In [ ]:
import gradio as gr
import time

def process_user_query(message, history):
    if len(history) % 2 == 0:
        return f"Yes, I do think that '{message}'"
    else:
        return "I don't think so"

def process_source_code(dir_path):
    progress = gr.Progress()
    # Simulate file processing with a delay
    for i in range(10):
        time.sleep(0.5)
        progress(i / 10)
        a = True

# Define Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# CodeDocBot")
    dir_path = gr.Textbox(label="Source-code directory path", placeholder="Enter the path to the folder")
    process_button = gr.Button("Process File")
    output_text = gr.Textbox(label="")
    
    process_button.click(fn=process_source_code, inputs=dir_path, outputs=output_text)

demo.launch()
gr.ChatInterface(process_user_query).launch()
    